# get_product_info

In [39]:
import datetime
import json
import os

import pandas as pd
import requests
from logzero import logger

from src.config import MAX_PAGE, REQ_URL, WANT_ITEMS, path_output_dir, req_params

In [25]:
brand = "margiela"
item = "財布"

In [26]:
cnt = 1
keyword = f"{brand} {item} 中古"

req_params["page"] = cnt
req_params["keyword"] = keyword
df = pd.DataFrame(columns=WANT_ITEMS)

In [27]:
# ページループ
logger.info("loop start!")
while True:
    req_params["page"] = cnt
    res = requests.get(REQ_URL, req_params)
    res_code = res.status_code
    res = json.loads(res.text)
    if res_code != 200:
        print(
            f"""
        ErrorCode -> {res_code}\n
        Error -> {res['error']}\n
        Page -> {cnt}"""
        )
    else:
        if res["hits"] == 0:
            print("返ってきた商品数の数が0なので、ループ終了")
            break
        tmp_df = pd.DataFrame(res["Items"])[WANT_ITEMS]
        df = pd.concat([df, tmp_df], ignore_index=True)
    if cnt == MAX_PAGE:
        print("MAX PAGEに到達したので、ループ終了")
        break
    break
    logger.info(f"{cnt} end!")
    cnt += 1
    # リクエスト制限回避
    sleep(1)

[I 240320 15:40:50 2510635517:2] loop start!


In [29]:
# df

In [32]:
print("Finished!!")
today = datetime.datetime.today().strftime("%Y%m%d")

Finished!!


In [33]:
# データフレームからリストを作成するための空のリスト
result_list = []

# データフレームの各行をループで処理
for index, row in df.iterrows():
    item_name = row["itemName"]
    price = row["itemPrice"]
    url = row["itemUrl"]
    result_list.append([item_name, price, url])

In [35]:
# result_list

In [40]:
# 結果のリストを表示
for i, data in enumerate(result_list):
    tweet_text = f"アイテム名: {data[0]}\n価格: {data[1]}\nURL: {data[2]}"
    # 本日日付フォルダ作成
    if not os.path.isdir(path_output_dir / brand):
        os.mkdir(path_output_dir / brand)
    # テキストファイルに書き込む
    with open(path_output_dir / brand / f"tweet_{today}_{i}.txt", "w") as file:
        file.write(tweet_text)

# auto_post_margiela

[X v2エンドポイントupdate後動作しなかったが、できるようになった参考](https://scr.marketing-wizard.biz/dev/tweepy-twitter-apiv2)

In [31]:
import os

import tweepy
from logzero import logger
from time import sleep
from src.config import TwitterAPI, path_output_dir

In [20]:
yyyymmdd = "20240320"
brand = "margiela"

In [26]:
client = tweepy.Client(
    bearer_token=TwitterAPI.BearerToken,
    consumer_key=TwitterAPI.Key,
    consumer_secret=TwitterAPI.KeySecret,
    access_token=TwitterAPI.AccessToken,
    access_token_secret=TwitterAPI.AccessTokenSecret,
)

In [21]:
def get_id_list(yyyymmdd: str, brand: str) -> list:
    return [
        int(f.split("_")[2].split(".txt")[0])
        for f in os.listdir(path_output_dir / brand)
        if f.startswith("tweet") and yyyymmdd in f
    ]

In [32]:
start_idx = min(get_id_list(yyyymmdd, brand))
end_idx = max(get_id_list(yyyymmdd, brand))
for idx in range(start_idx, end_idx + 1):
    logger.info(f"auto load tweet:{idx}")
    try:
        with open(path_output_dir / brand / f"tweet_{yyyymmdd}_{idx}.txt", "r") as f:
            try:
                client.create_tweet(text=f.read())
                sleep(1)
            except tweepy.errors.Forbidden as e:
                logger.info(f"{e}")
    except FileNotFoundError:
        logger.info(f"file not found:{idx}")
        continue

[I 240320 16:15:20 3158587560:4] auto load tweet:0
[I 240320 16:15:21 3158587560:11] 403 Forbidden
    You are not allowed to create a Tweet with duplicate content.
[I 240320 16:15:21 3158587560:4] auto load tweet:1
[I 240320 16:15:22 3158587560:4] auto load tweet:2
[I 240320 16:15:23 3158587560:4] auto load tweet:3
[I 240320 16:15:25 3158587560:4] auto load tweet:4
[I 240320 16:15:26 3158587560:4] auto load tweet:5
[I 240320 16:15:27 3158587560:4] auto load tweet:6
[I 240320 16:15:29 3158587560:4] auto load tweet:7
[I 240320 16:15:30 3158587560:4] auto load tweet:8
[I 240320 16:15:32 3158587560:4] auto load tweet:9
[I 240320 16:15:33 3158587560:4] auto load tweet:10
[I 240320 16:15:34 3158587560:4] auto load tweet:11
[I 240320 16:15:36 3158587560:4] auto load tweet:12
[I 240320 16:15:37 3158587560:4] auto load tweet:13
[I 240320 16:15:38 3158587560:4] auto load tweet:14
[I 240320 16:15:40 3158587560:4] auto load tweet:15
[I 240320 16:15:41 3158587560:4] auto load tweet:16
[I 240320 16